In [1]:
# 이 부분은 처음 한번만 실행하면 됌.
# 코드 수정 - "The reason is that the last Ubuntu update update supports chromium driver just via snap."
# 최근 우분투 업데이트에서 크롬 드라이버 설치를 snap을 이용해서만 하도록 바뀜
# 고로 snap 없이 설치하는 아래 우회 코드로 변경
# 출처 : https://colab.research.google.com/drive/1cbEvuZOhkouYLda3RqiwtbM-o9hxGLyC
# 출처2 : https://stackoverflow.com/questions/75155063/selenium-use-chrome-on-colab-got-unexpectedly-exited

%%shell
# Ubuntu no longer distributes chromium-browser outside of snap
#
# Proposed solution: https://askubuntu.com/questions/1204571/how-to-install-chromium-without-snap

# Add debian buster
cat > /etc/apt/sources.list.d/debian.list <<'EOF'
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-buster.gpg] http://deb.debian.org/debian buster main
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-buster-updates.gpg] http://deb.debian.org/debian buster-updates main
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-security-buster.gpg] http://deb.debian.org/debian-security buster/updates main
EOF

# Add keys
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys DCC9EFBF77E11517
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys 648ACFD622F3D138
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys 112695A0E562B32A

apt-key export 77E11517 | gpg --dearmour -o /usr/share/keyrings/debian-buster.gpg
apt-key export 22F3D138 | gpg --dearmour -o /usr/share/keyrings/debian-buster-updates.gpg
apt-key export E562B32A | gpg --dearmour -o /usr/share/keyrings/debian-security-buster.gpg

# Prefer debian repo for chromium* packages only
# Note the double-blank lines between entries
cat > /etc/apt/preferences.d/chromium.pref << 'EOF'
Package: *
Pin: release a=eoan
Pin-Priority: 500


Package: *
Pin: origin "deb.debian.org"
Pin-Priority: 300


Package: chromium*
Pin: origin "deb.debian.org"
Pin-Priority: 700
EOF

# Install chromium and chromium-driver
apt-get update
apt-get install chromium chromium-driver


# Install selenium
pip install selenium
pip install webdriver_manager 
pip install pymongo
pip install newspaper3k

Executing: /tmp/apt-key-gpghome.fH3nD5pDKc/gpg.1.sh --keyserver keyserver.ubuntu.com --recv-keys DCC9EFBF77E11517
gpg: key DCC9EFBF77E11517: public key "Debian Stable Release Key (10/buster) <debian-release@lists.debian.org>" imported
gpg: Total number processed: 1
gpg:               imported: 1
Executing: /tmp/apt-key-gpghome.jFIONpVpIW/gpg.1.sh --keyserver keyserver.ubuntu.com --recv-keys 648ACFD622F3D138
gpg: key DC30D7C23CBBABEE: public key "Debian Archive Automatic Signing Key (10/buster) <ftpmaster@debian.org>" imported
gpg: Total number processed: 1
gpg:               imported: 1
Executing: /tmp/apt-key-gpghome.fOa9LpzMCv/gpg.1.sh --keyserver keyserver.ubuntu.com --recv-keys 112695A0E562B32A
gpg: key 4DFAB270CAA96DFA: public key "Debian Security Archive Automatic Signing Key (10/buster) <ftpmaster@debian.org>" imported
gpg: Total number processed: 1
gpg:               imported: 1
Get:1 http://deb.debian.org/debian buster InRelease [122 kB]
Get:2 http://deb.debian.org/debian bust

In [ ]:
# 셀레니움 module import 
# from selenium import webdriver
# 2023.05.08 selenium webdriver 사용방법 변경 
from selenium.webdriver.common.by import By
from selenium import webdriver
import time 
from time import sleep
from bs4 import BeautifulSoup
from selenium.webdriver.common.keys import Keys
import requests
import re
import os, sys
from bs4 import BeautifulSoup
import requests
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
import time
import json
from tqdm import tqdm
from newspaper import Article
import pymongo

chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('disable-dev-shm-usage')
chrome_options.add_argument("user-agent=Mozilla/5.0 (Macintosh; Intel Mac OS X 10_12_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/61.0.3163.100 Safari/537.36")
driver = webdriver.Chrome('chromedriver', options=chrome_options)

!pip3 install apscheduler

import time
from datetime import date
from apscheduler.schedulers.blocking import BlockingScheduler


def get_html (src, output): 
    chrome_options = webdriver.ChromeOptions()
    chrome_options.add_argument('--headless')
    chrome_options.add_argument('--no-sandbox')
    chrome_options.add_argument('disable-dev-shm-usage')
    chrome_options.add_argument("user-agent=Mozilla/5.0 (Macintosh; Intel Mac OS X 10_12_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/61.0.3163.100 Safari/537.36")
    driver = webdriver.Chrome('chromedriver', options=chrome_options)
    # driver = webdriver.Chrome()

    driver.get(src)

    new_height = driver.execute_script("return window.pageYOffset")
    prev_height = driver.execute_script("return document.body.scrollHeight")

    while True:
        driver.execute_script("window.scrollTo(0,{})".format(new_height + 50000))
        time.sleep(3)
        new_height = driver.execute_script("return window.pageYOffset")
        if prev_height == new_height: break
        else: prev_height = new_height

    with open(output, 'w') as f:
        f.write(driver.page_source)
    driver.close()


def get_url(file, src):
    cont = open(file, 'r')
    html = BeautifulSoup(cont, 'html.parser')
    urlset = []
    for i in range(1, 173):
        selector = "#screenerDetail-0-Stream > ul > li:nth-child("+str(i)+") > div > div > div > h3 >a"
        item = html.select(selector)
        if item != [] : 
            url = item[0].attrs['href']
            if 'beap.gemini' in url: continue #skip ads
            else:
                srcurl = src + url
                urlset.append(srcurl)
        else : break
    print('number of articles..', len(urlset))

    return urlset


def get_news_full(url):
    article = Article(url, language='en')
    article.download()
    article.parse()
    return article.text


def get_news(urlset,article_db):
    if urlset == []:
        return 'url not exist'
    
    total_news = []
    headers = { "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/110.0.0.0 Safari/537.36" }
    for url in urlset:
        news={}

        try:
            raw = requests.get(url,headers=headers)
            html = BeautifulSoup(raw.text, "html.parser")
        except requests.exceptions.RequestException as e:
            print("Error occurred: ", e)
            continue
        
        try : html.select('#module-article > div')[0]
        except IndexError:
            print(url)
            break

        newsid = html.select('#module-article > div')[0].attrs['id']
        info = html.select("#{} > article > script:nth-child(1)".format(newsid))
        content = json.loads(info[0].contents[0])
        title = content['headline']
        keyword = content['keywords']
        provider = content['provider']['name']
        datepublished = content['datePublished']
        body = get_news_full(url)
        news['title'] = title
        news['keyword'] = keyword
        news['provider'] = provider
        news['datepublished'] = datepublished
        news['content'] = body
        total_news.append(news)

        article_db.insert_one(news)  

    return total_news


def save_json(total_news, final):
       
    for news1 in total_news:
        try :
          news1.pop("_id")
        except :
          continue
    
    with open(final, 'w', encoding='utf-8') as f:
        json.dump(total_news, f, ensure_ascii=False, indent='\t')



def main():
    industries = ['ms_basic_materials', 'ms_communication_services', 'ms_consumer_cyclical', 'ms_consumer_defensive', 
              'ms_energy', 'ms_financial_services', 'ms_healthcare', 'ms_industrials', 'ms_real_estate', 'ms_technology', 'ms_utilities']
    
    # set path for html and json output
    html_path = '/content/drive/MyDrive/ARA/crawled_html/'
    output_path = '/content/drive/MyDrive/ARA/crawled_article/'
    
    for industry in tqdm(industries):
        src1 = "https://finance.yahoo.com/screener/predefined/"+industry+"/"
        src2 = "https://finance.yahoo.com/news/"
        output = industry+".html"
        final = industry+'_news.json'

        db_url = 'mongodb://ara-prod:LDBvT8AQvOdptEbDX2GU8sAL5ogUzDqpdvCHPQJgfSMmfLPqhWoLTXvuY8DTE0PPMuCLwOMgyGczACDbUJztYw==@ara-prod.mongo.cosmos.azure.com:10255/?ssl=true&retrywrites=false&replicaSet=globaldb&maxIdleTimeMS=120000&appName=@ara-prod@'
        DB_NAME = 'ARA_DB'

        client = pymongo.MongoClient(db_url)
        db = client[DB_NAME]

        if DB_NAME not in client.list_database_names():
            # Create a database with 400 RU throughput that can be shared across
            # the DB's collections
            db.command({"customAction": "CreateDatabase", "offerThroughput": 400})
            print("Created db '{}' with shared throughput.\n".format(DB_NAME))
        else:
            print("Using database: '{}'.\n".format(DB_NAME))

        ARTICLE_COLLECTION = industry.upper()
        article_db = db[ARTICLE_COLLECTION]

        if ARTICLE_COLLECTION not in db.list_collection_names():
            # Creates a unsharded collection that uses the DBs shared throughput
            db.command(
                {"customAction": "CreateCollection", "collection": ARTICLE_COLLECTION}
            )
            print("Created collection '{}'.\n".format(ARTICLE_COLLECTION))
        else:
            print("Using collection: '{}'.\n".format(ARTICLE_COLLECTION))

        
        get_html (src1, html_path+output)
        urlset = get_url(html_path+output, src2)
        print('Starts getting articles for ', industry)
        total_news = get_news(urlset, article_db)
        save_json(total_news, output_path+final)

if __name__ == '__main__':
    main()

# sched = BlockingScheduler(timezone='Asia/Seoul')
# @sched.scheduled_job('cron', hour='16', minute='39', id='test_1') # 매일 5시 30분에 실행
# def job1():
#     print(f'daily job started for : {date.today()}')
#     main()

# print('sched before~')
# sched.start()

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


  0%|          | 0/11 [00:00<?, ?it/s]

Using database: 'ARA_DB'.

Using collection: 'MS_BASIC_MATERIALS'.

number of articles.. 137
Starts getting articles for  ms_basic_materials


  9%|▉         | 1/11 [05:39<56:38, 339.85s/it]

Using database: 'ARA_DB'.

Using collection: 'MS_COMMUNICATION_SERVICES'.

number of articles.. 137
Starts getting articles for  ms_communication_services


...

Using database: 'ARA_DB'.

Using collection: 'MS_INDUSTRIALS'.

number of articles.. 137
Starts getting articles for  ms_industrials


 73%|███████▎  | 8/11 [41:57<15:46, 315.49s/it]

Using database: 'ARA_DB'.

Using collection: 'MS_REAL_ESTATE'.

number of articles.. 137
Starts getting articles for  ms_real_estate


 82%|████████▏ | 9/11 [48:05<11:03, 331.97s/it]

Using database: 'ARA_DB'.

Using collection: 'MS_TECHNOLOGY'.

number of articles.. 139
Starts getting articles for  ms_technology


In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

In [ ]:
# !pip install pymongo
# import pymongo

# db_url = 'mongodb://ara-prod:LDBvT8AQvOdptEbDX2GU8sAL5ogUzDqpdvCHPQJgfSMmfLPqhWoLTXvuY8DTE0PPMuCLwOMgyGczACDbUJztYw==@ara-prod.mongo.cosmos.azure.com:10255/?ssl=true&retrywrites=false&replicaSet=globaldb&maxIdleTimeMS=120000&appName=@ara-prod@'
# DB_NAME = 'ARA_DB'

# FEEDBACK_COLLECTION = 'FEEDBACK'

# client = pymongo.MongoClient(db_url)
# db = client[DB_NAME]

# feedback_db = db[FEEDBACK_COLLECTION]

# industries = ['ms_basic_materials', 'ms_communication_services', 'ms_consumer_cyclical', 'ms_consumer_defensive', 
#           'ms_energy', 'ms_financial_services', 'ms_healthcare', 'ms_industrials', 'ms_real_estate', 'ms_technology', 'ms_utilities']

# ARTICLE_COLLECTION = industries[0].upper()
# article_db = db[ARTICLE_COLLECTION]
# articles = article_db.find({},{'_id':0,'content':1}) #content만 가져오기 (가져올 column : 1, 안 가져올 column : 0)

# article_db.count_documents({}) #document 갯수 count
# articles[0]